## **演示1204：随机森林**

### **基本原理**
* 单一决策树很容易产生过拟合；如果采用多棵决策树，共同投票来做决定，往往会比采用单一决策树具有更好的效果。
* Bagging策略：从样本集（假设样本集$N$个数据点）中重采样选出$n$个样本（有放回的采样，样本数据点个数仍然不变为$N$），对这$n$个样本建立分类器（ID3\C4.5\CART等方法），重复以上两步$m$次，获得$m$个分类器，最后根据这$m$个分类器的投票结果，决定数据属于哪一类。
* 使用步骤：
 * 样本的随机：从样本集中用Bagging策略随机选取$n$个样本
 * 特征的随机：从所有属性中随机选取$K$个属性，选择最佳分割属性作为节点建立CART决策树（也可以是其他类型的分类器，比如SVM、Logistics）
 * 重复以上两步$ m $次，即建立了$ m $ 棵CART决策树
 * 这$m$个CART形成随机森林，通过投票表决结果，决定数据属于哪一类（投票机制有一票否决制、少数服从多数、加权多数）

### **案例1：使用自定义随机森林分类器**
* 分类器代码参考【random_forest.py】
* 生成多棵决策树
 * 每棵决策树使用不完全相同的样本数据。每次生成决策树之前，先将原始样本数据的顺序打散，然后取前若干条数据作为样本
 * 因为特征数量不多，因此本例没有对特征进行随机选取，而是直接使用了所有特征
* 投票
 * random_forest.forest_classify方法用于在多个决策树之间做出判别并根据投票结果给出最终判别

In [1]:
''' 基于ID3决策树的随机森林实现 '''
import numpy as np    
import collections as col
import random_forest

data   = []    
labels = []   
inputs = [] 
with open("car.csv") as ifile:    
        first_line = True
        for line in ifile:
            if first_line:              # 跳过第一行(标题行)
                first_line = False
                continue
            rowDict = {}
            tokens = line.strip().split(',')  
            rowDict['buying']=tokens[0]
            rowDict['maint']=tokens[1]  
            rowDict['doors']=tokens[2]  
            rowDict['persons']=tokens[3]  
            rowDict['lug_boot']=tokens[4]  
            rowDict['safety']=tokens[5]  
            inputs.append((rowDict, False if tokens[-1]=='unacc' else True))

total_count = len(inputs)
train_inputs = []
test_inputs = []
temp = train_inputs, test_inputs
ratio = 0.75
for i in range(len(inputs)):
    dataSetIndex = 0 if np.random.random() < ratio else 1
    temp[dataSetIndex].append(inputs[i])

tree_count = 6              # 生成6棵树
sample_ratio = 0.75         # 每棵树中样本比例
sample_count = int(sample_ratio*len(train_inputs))
trees = []
for i in np.arange(tree_count):
    np.random.shuffle(train_inputs)                 # 打乱样本
    sample_inputs = train_inputs[:sample_count]
    tree = random_forest.build_tree_id3(sample_inputs)
    trees.append(tree)

correct_count = 0
for row in test_inputs:
    predict = random_forest.forest_classify(trees, row[0])
    if predict == row[1]:
        correct_count += 1
print("预测正确率：", correct_count / len(test_inputs))

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.
预测正确率： 0.9549763033175356


### **案例2：使用sklearn.ensemble.RandomForestClassifier**

In [2]:
''' 使用RandomForestClassifier '''
import numpy as np    
from sklearn.feature_extraction import DictVectorizer 
from sklearn.ensemble import RandomForestClassifier

data   = []    
labels = []    
with open("car.csv") as ifile:    
        for line in ifile:
            #data需要是字典形式，因为之后需要使用DictVectorizer()修改字符串数据类型，以便符合DecisionTreeClassifier()  
            rowDict = {}
            tokens = line.strip().split(',')  
            rowDict['buying']=tokens[0]
            rowDict['maint']=tokens[1]  
            rowDict['doors']=tokens[2]  
            rowDict['persons']=tokens[3]  
            rowDict['lug_boot']=tokens[4]  
            rowDict['safety']=tokens[5]  
            data.append(rowDict)  
            labels.append(tokens[-1])   
x = np.array(data)  
labels = np.array(labels)    
y = np.zeros(labels.shape)  # 初始label全为0  
  
y[labels =='vgood']=1       # 当label等于这三种属性的话，设置为1。  
y[labels =='good']=1  
y[labels =='acc']=1  
  
vec = DictVectorizer()      # 转换字符串数据类型  
dx = vec.fit_transform(x).toarray()  

# 拆分成训练数据和测试数据
ratio = 0.75
xTrain = []
yTrain = []
xTest = []
yTest = []
features = xTrain,xTest
labels = yTrain, yTest
for i in range(len(dx)):
    dataSetIndex = 0 if np.random.random() < ratio else 1
    features[dataSetIndex].append(dx[i])
    labels[dataSetIndex].append(y[i])
  
clf = RandomForestClassifier()
clf.fit(xTrain,yTrain)        

# 检查准确率
accuracy = clf.score(xTest, yTest)
print(accuracy)

0.9875311720698254


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
